# Load Labels

In [1]:
from tqdm import tqdm_notebook as tqdm

In [2]:
twitter15_label_file = '../twitter15/label.txt'
twitter15_text_file = '../twitter15/source_tweets.txt'

In [3]:
def load_labels(file):
    f = open(file,'r')
    labels = {}
    
    raw_data = f.readlines()
    
    for line in raw_data:
        line = line.strip()
        line = line.split(':')
        labels[int(line[1])] = line[0]
    
    return labels

In [4]:
twitter15_labels = load_labels(twitter15_label_file)

In [5]:
twitter15_labels

{731166399389962242: 'unverified',
 714598641827246081: 'unverified',
 691809004356501505: 'non-rumor',
 693204708933160960: 'non-rumor',
 551099691702956032: 'true',
 767223774072676354: 'non-rumor',
 715515982584881152: 'unverified',
 514106273852174337: 'true',
 500319801344929795: 'unverified',
 495366618818830336: 'false',
 532206910796468224: 'false',
 560187970389819392: 'false',
 531568534066057217: 'false',
 489829414704648192: 'false',
 524925730053181440: 'unverified',
 766989078294306816: 'non-rumor',
 499530130487017472: 'unverified',
 520284654755381249: 'false',
 767515401831997440: 'non-rumor',
 565999191982616577: 'true',
 554343513887105024: 'false',
 767715489485205505: 'non-rumor',
 553467311261503488: 'unverified',
 553960736964476928: 'false',
 500303431928922113: 'unverified',
 538900739880665088: 'unverified',
 516420964834611201: 'unverified',
 80080680482123777: 'false',
 687945926774800385: 'non-rumor',
 436089462326849536: 'false',
 568589712644026368: 'fals

# Load Tweet Texts

In [6]:
twitter15_text = {}
def load_text(file):
    f = open(file,'r')
    labels = {}
    
    raw_data = f.readlines()
    
    for line in raw_data:
        line = line.strip()
        line = line.split('\t')
#         print(line[0],line[1])
        labels[int(line[0])] = line[1]
    
    return labels

In [7]:
twitter15_text = load_text(twitter15_text_file)

In [8]:
twitter15_text

{731166399389962242: '🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL',
 714598641827246081: 'an open letter to trump voters from his top strategist-turned-defector URL via @xojanedotcom',
 691809004356501505: '"america is a nation of second chances" —@potus on new reforms to solitary confinement: URL URL',
 693204708933160960: 'brandon marshall visits and offers advice, support to brother of fallen hero zaevion dobson: URL URL',
 551099691702956032: 'rip elly may clampett: so sad to learn #beverlyhillbillies star donna douglas has passed away. URL',
 767223774072676354: 'former 3 doors down guitarist matt roberts has died at age 38, according to his father. URL URL',
 715515982584881152: 'craigslist ad: ‘get paid $15 an hour to protest at the trump rally’ - URL URL',
 514106273852174337: 'just in: missing afghan soldiers found trying to enter canada near niagara falls URL URL',
 500319801344929795: 'the day #ferguson cops told a dirty, bloody lie (via @the

# Data Structure For Storing Tree Structures

In [9]:
from __future__ import print_function

In [10]:
class Node:
    def __init__(self,uid,tid,time_stamp):
        self.children = {}
        self.tid = tid
        self.uid = uid
        self.time_stamp = time_stamp
        
        self.branching = 0
    
    def add_child(self,node):
        self.children[node.uid] = node
        self.branching += 1

In [11]:
class Tree:
    def __init__(self,root):
        self.root = root
        self.tweet_id = root.tid
        self.uid = root.uid
        self.height = 0
        self.nodes = 0
    
    def show(self):
        queue = [self.root,0]
        
        while len(queue) != 0:
            toprint = queue.pop(0)
            if toprint == 0:
                print('\n')
            else:
                print(toprint.uid,end=' ')
                queue += toprint.children.values()
                queue.append(0)

# Get User Information from API

In [12]:
import sys
import tweepy
import re

In [13]:
import pickle as pkl
from collections import defaultdict

In [ ]:
consumer_key="LydBaZlcnj64wAhz2zqYSSDG8"
consumer_secret="lNDf9f9xcz3qjbXVxcKaF65t8vEvwpDlRQbljwLWS6NOst0rWb"
access_key="747987764579176448-TmAC6q6EYzK2czKk2wpwSBCycYFjLpd"
access_secret="PHaIgoAFNZVdocFVepu3MwrMauzWlDSrTjVBUSrcpUTcD"

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [ ]:
def get_userinfo(uid):
    user = api.get_user(uid)
    return user

In [14]:
user_count = defaultdict(lambda: 0)
user_data = {}
user_features = {}

# Load Trees

In [15]:
from os import listdir
from os.path import isfile, join
import re
import numpy as np

In [16]:
datapath = '../twitter15/tree/'
treefiles = [f for f in listdir(datapath) if isfile(join(datapath, f))]

In [17]:
def insertnode(tree,curnode,parent,child):
#     print(curnode.uid,parent.uid,child.uid)
    if curnode.uid == parent.uid:
        curnode.add_child(child)
        return 1
    
    elif parent.uid in curnode.children:
        s = insertnode(tree,curnode.children[parent.uid],parent,child)
        return 2
    else:
        for node in curnode.children:
            s = insertnode(tree,curnode.children[node],parent,child)
            if s == 2:
                break

In [18]:
def file2tree(datapath,txtfile):
    f = open(datapath+txtfile,'r')
    rawtree = f.readlines()
    tree = None
    count = 0
    nodes = {}
    root = -1
    
    for line in rawtree:
        count += 1
        node1 = None
        
        line = line.strip()
        edge = re.split('\]->\[',line)
        edge[0] = edge[0][1:].split(', ')
        edge[1] = edge[1][:-1].split(', ')
        
        edge[0] = [x[1:-1] for x in edge[0]]
        edge[1] = [x[1:-1] for x in edge[1]]
        
        edge[1][0] = int(edge[1][0])
        edge[1][1] = int(edge[1][1])
        edge[1][2] = float(edge[1][2])
        
        user_count[edge[1][0]] += 1
        
        node2 = Node(edge[1][0],edge[1][1],edge[1][2])
        
        if edge[0][1][0] != 'R':
            edge[0][0] = int(edge[0][0])
            edge[0][1] = int(edge[0][1])
            edge[0][2] = float(edge[0][2])
            
#             print(edge)
            
            node1 = Node(edge[0][0],edge[0][1],edge[0][2])
            insertnode(tree,root,node1,node2)
            tree.nodes += 1
            
        else:
            root = node2
            nodes[node2.uid] = node2
            tree = Tree(root)
            tree.nodes += 1
    
    return tree

In [19]:
def check_children(node,h):
    branching_factor = []
#     print(node.uid)
    
    for key in node.children:
#         print(node.branching)
        child = node.children[key]
        if child.branching > 0:
            branching_factor.append(1)
            check_children(child,h)
            
        else:
            branching_factor.append(0)
            
    branching_factor = np.asarray(branching_factor)
    
    if np.any(branching_factor):
        h += 1
        
    return h

In [20]:
def get_treeheight(tree):
    h = check_children(tree.root,2)
    return h

In [21]:
twitter15_trees = []
for treefile in tqdm(treefiles):
    try:
        tree = file2tree(datapath,treefile)
        tree.height = get_treeheight(tree)

        twitter15_trees.append(tree)
    except:
        print(treefile)

516371936604352513.txt
766808183696351233.txt
519868410599993344.txt
514517213022543872.txt
531607884220485632.txt
523123779124600833.txt
693729678306189313.txt
693631772639145984.txt
745365403237376000.txt
689719109785907200.txt
550175324634619904.txt
528822281972498432.txt
689722724470620161.txt
538975342011363328.txt
752965545528528898.txt
519929168864497664.txt
715255507506892800.txt



In [ ]:
twitter15_trees

## Basic Stats

In [22]:
avght = 0
avgnodes = 0
for tree in twitter15_trees:
    avght += tree.height
    avgnodes += tree.nodes
avght = avght/len(twitter15_trees)
avgnodes = avgnodes/len(twitter15_trees)

In [23]:
avght

2.9966055668703326

In [24]:
avgnodes

417.04684317718943

In [25]:
user_count

defaultdict(<function __main__.<lambda>()>,
            {30313925: 42,
             113351692: 1,
             147657826: 1,
             285962772: 2,
             233033031: 6,
             75649353: 22,
             301692966: 20,
             20715638: 81,
             602989819: 3,
             565623639: 3,
             716709528: 10,
             729789925: 1,
             311074587: 5,
             170421988: 1,
             66753468: 1,
             201129833: 51,
             198186350: 1,
             3395568441: 1,
             1264291556: 6,
             1155531206: 1,
             1447442666: 1,
             2863510746: 5,
             276239681: 2,
             127643327: 7,
             2279779670: 1,
             2416436780: 6,
             315858046: 2,
             29315281: 1,
             2787473801: 7,
             926580307: 1,
             4798011916: 13,
             406402277: 1,
             55420724: 5,
             20464641: 1,
             47403889: 16,
  

In [26]:
count = 0
for key in tqdm(user_count):
    try:
        if key not in user_data:
            data,vect = get_userinfo(key)
            user_data[key] = data
            user_features[key] = vect
    except:
        count += 1
        print(key)

30313925
113351692
147657826
285962772
233033031
75649353
301692966
20715638
602989819
565623639
716709528
729789925
311074587
170421988
66753468
201129833
198186350
3395568441
1264291556
1155531206
1447442666
2863510746
276239681
127643327
2279779670
2416436780
315858046
29315281
2787473801
926580307
4798011916
406402277
55420724
20464641
47403889
2844276451
3141913771
239682170
2800811105
14890490
2203880112
1296678864
49074629
3122114718
3252365815
44696044
4364389877
86360236
1345932331
4691537418
3067136934
4615927709
983281712
3238370479
3186961866
41435071
275402198
4616897243
273691052
425192351
20771915
276564736
30411850
971131868
15819729
95602718
461628773
300418862
302911198
157049291
382373735
2905536316
32944248
4243275815
2450106906
4060311261
273397369
295712521
562191895
57930693
358140803
478471916
3192377917
3951098415
1660624110
528842323
197233020
429382552
220564852
15547596
18394497
173968152
1576771369
75640189
21402164
1593904718
21430391
4827846977
554514035


775966501
482644250
18233608
490909356
711801073
48481482
1716223568
75721718
1338128491
58704022
403293302
2193594721
2831491969
277049504
2892376188
966046099
442456353
760427960
16496337
164411930
256837356
505425467
554635643
2958499297
701237714453143552
39837049
357950759
2353178919
229334567
220566302
740446116693614592
78718212
291405122
10985382
397726501
4787468909
309757145
540146924
271953315
112001355
94209518
742085156
118403531
239546439
296846453
2449862432
530994576
518974548
2380987147
1628075760
609374869
2232934945
1547927492
159837500
718637930
2747526438
757146343131508736
766611261702438912
22704101
2564198618
4850263133
300600995
170440395
266161551
3065622753
3424723883
19855158
2697913410
341538016
240707868
2617795268
718811790897848320
2986870187
165484106
16090774
3298295492
765709971594018816
305305516
406471326
3975756309
55400718
290225617
21469013
163426868
110241573
1038075991
756276116810194944
2491352002
383607345
451273707
267179135
7566487286777118

778339871374966784
4196941433
2467234817
3304987564
2320019451
1957298564
337090982
1656591433
2183341540
3279152940
324709343
3232831610
529677813
2975758635
2485227765
515278972
2392838779
2676646846
2281373516
621953127
4821797819
2262451602
3180323305
2986408619
877730935
930359917
1663299422
3436869838
4474753882
193113108
735353191
839664684
1676786358
3692581097
2542139669
4672496660
4489122444
1252237940
805377882
2296155629
1458803678
4375299748
3226370422
3136180518
2362995882
2761668775
3117673415
2723662055
4139924473
2650174936
2980006710
3926395214
939332077
2990374064
2741537991
876362582
1658639310
1654713049
4817665589
4850484916
2619393005
114325910
3711631659
3048037004
3997169578
488884556
894573901
1325394578
2255737710
2755744068
2534566679
3280827180
2869823719
2382547123
3570152117
2952794592
2258586789
4774533093
2278167198
2230380568
2964174570
375267061
2609515965
3214710944
2272331990
1635613422
4724813825
2886071617
2587063832
2355207471
3253022290
28327831

153995334
38292041
915860389
63935448
177087665
570360944
25779637
577835829
850318543
2794679030
1533267187
381355445
182084409
353403242
16618860
211523238
76881390
177611310
31268814
302023808
113744743
769702230
466010965
876008257
16006466
216786228
548889337
17719592
1323219212
19052381
560409728
378702565
368523309
626601511
1669056901
1576434258
390520212
108736723
272761222
2474695363
2800808066
15959570
2155855183
1219034761
1463236442
265051635
894668179
134345765
919069508
90961739
340709125
387572703
1747131282
359496651
863755201
37435889
31496604
1099430960
219434634
618275847
410888083
236209053
2259507037
2230682301
192861406
75807702
316161690
519076313
32785658
129258722
68689143
472161328
27867068
35318715
2733112614
2441379877
15951166
223162337
838157258
246545966
31204212
283823914
289987170
2390092718
995469566
552177847
410489408
633394796
1850146886
221445245
2516864490
164628686
888572174
165161481
2731068386
2249500333
2370724255
104262985
297055396
22847135

1190940379
1514468359
118949926
1447746264
786094454
579428567
297716123
464651414
92196565
14980181
50839450
27508594
113781031
960986761
42481125
25831235
29334203
527247484
935486508
25952449
429488854
4616875155
153865388
17842534
1276116846
180134634
285324937
16282330
246093025
38668948
631782145
221952179
405196895
1834798070
3252965044
61734492
236395477
1480123032
390669478
14182589
292072162
300369016
26410254
4724701032
51761095
19477705
19253732
165613572
62646197
602601585
480819613
27034859
863845507
201709979
16490318
1730088331
1674114042
356398291
2261649066
3721903880
1630796316
236843273
44732668
291780567
177088414
265585421
119568024
152873197
31280052
408049631
192242727
493731014
1207419391
361369415
3441403815
2784863960
179780556
1195009705
466379654
41246537
499825197
2774744636
50712392
18713335
56000803
45762728
386672549
1379754139
1963613821
74500005
316044040
258517413
249004971
607527343
128033744
285293911
1063737282
424490650
155402343
447561388
468601

...

1339032516
239965190
4768143821
2897073229
4849444011
1273576430
1937835708
554058160
3325433880
277602640
987483512
28519044
77092508
193262029
37540556
2548183052
480075222
490793643
565147022
352684925
417490754
2274546031
1849717556
27142630
2548839590
598713333
64718189
21377106
499955286
61823325
3861049394
77245525
58092038
36287184
30894725
1399037323
271578675
199527887
2853457409
556118703
4546398514
193867940
2441228080
246619709
452196244
360056181
452521686
832022714
393559036
20484508
34939367
1557989598
145675480
1702781406
2213930756
2701881606
815569712
1669559376
755396346
30306325
1532827476
622937885
95712551
43751916
99444811
458776654
261576484
17390306
388681921
2822417315
50493671
3707821032
362011115
1315214048
16417288
558215848
3338478090
96319046
423767216
2781777808
499937529
3368046069
3010635690
431753036
26409669
2732024969
1848480360
830939480
386556932
369098181
19965430
863628498
3696730994
849862741
2842740756
1547356278
15103941
39522974
2993587478


844810166
135449306
111307250
414947974
1162648014
2885834010
2653924749
1911208674
1410647059
795955172
167860942
1152377454
179442187
495085684
2456969885
40037099
337641377
2539207590
1480273399
46625958
2281421358
636367254
2554022491
143135729
490931304
2848478863
604854665
166445279
4855687055
616871283
179250417
265245636
1400943320
2984517453
281551999
408949404
637096765
609109916
707310778055929856
738504686123126785
4862157717
339820844
896601648
3422158725
2966724502
3126563358
4268991619
2595978877
62493973
3407711843
4895096885
704303044662439936
2377705375
2863451998
4892318567
1263033402
3306237726
1416794803
750561204905672704
4229255592
720927815206641664
2271813558
761380526
3193563058
188425645
735524232948486145
762488659966775298
28273116
1714215379
66819988
2355934348
4874948752
3072832073
4861558613
3280073143
4061705121
81189820
2262620679
856800361
23691853
1647839894
867347492
706105772048908288
4838753877
746482529436262400
4806311422
709462701693493248
1722

2737574550
178207796
270176020
3420581463
1562026968
3379078817
602300975
4032985869
3315335401
3179820309
3041842986
3092806751
3227823331
3429873279
3237041161
2769216951
4029936252
718086045
3401978362
3032044047
2894150756
4767248602
3092599289
1382480527
3014919107
3397255932
1862410568
3105300581
4841733994
2937018602
557211008
300190431
3224261300
4804052623
3304732351
1337714178
405090398
723348722
2172440958
4273759066
283766020
24492629
2860544959
1538605603
1850142211
1504882254
2984798465
2201738896
3288976463
3197781366
3643908082
4706977762
4827308031
30798886
3008387937
3291116797
3135393622
3349095593
2174498936
3377285697
3312258269
709386475
2155318243
3709411392
3039700467
210650406
4848354904
2848190853
4201434258
3959797829
415778773
2869372942
4615969815
84533354
4788637580
1719330612
2191115017
2938622692
750380238
3295976413
2750418385
998810924
4719411325
848144610
2828611793
438546752
429653369
4330730308
4831727791
3399116512
3695873894
2380641878
122665394
1

611060199
526463447
1604333419
232078786
29681291
604907492
2344284739
2241736868
20686232
403088821
2415732613
26519432
463389676
1648593704
418024771
2213292602
1360721107
1465133910
440285897
2484982652
234081232
214547779
443374695
18526983
1920954174
242329918
41702521
1183009507
2743585979
2255610974
246373278
612596539
862626912
183068111
1563050532
534879197
456135883
24165761
1196584692
88602886
27661137
217570329
45865590
768572234
388440709
715478815
211752660
254624520
73267432
555256262
2568512598
119512960
1362603362
16910131
456665228
100335279
90536965
33234027
80230331
311870165
305087933
1567260500
514868149
24500370
57068980
26602717
24588844
262713997
310673681
94618485
579156366
2706891674
325109269
1357424546
245888455
1565933694
308735907
342750375
433735300
143968426
225973025
371982069
33558270
1095005778
451868256
312288868
53059640
205309206
223664682
83039463
1525208162
263020095
87044545
17852703
18473742
156091424
433917109
798442806
189304379
15970073
192

KeyboardInterrupt: 

In [28]:
len(user_count)

474469

In [27]:
twitter15_trees[1].tweet_id

515519742355185664

In [ ]:
import pickle

In [ ]:
f = open('user_ids.pkl','wb+')
pickle.dump(list(user_count.keys()),f)
f.close()